# Lysozyme + Mutations tutorial using BIOBB
Based on the official Gromacs tutorial: http://www.mdtutorials.com/gmx/lysozyme/01_pdb2gmx.html

In [3]:
import nglview
import ipywidgets

## Dowload the initial Structure
Download the initial PDB file from a PDB code (1AKI) from the RCSB (https://files.rcsb.org/download/).
The PDB will be dowloaded without solvent or HETATM.

In [4]:
#Import module
from biobb_io.api.pdb import Pdb
# Create properties dict and inputs/outputs
prop = {'pdb_code':'1aki'}
downloaded_pdb_path = '1aki.pdb'
#Create and launch bb
Pdb(output_pdb_path=downloaded_pdb_path, properties=prop).launch()

2019-03-06 16:53:22,139 [MainThread  ] [INFO ]  Downloading: 1aki from: https://files.rcsb.org/download/1aki.pdb
2019-03-06 16:53:22,833 [MainThread  ] [INFO ]  Writting pdb to: /Users/pau/projects/biobb_wf_mutations/biobb_wf_mutations/notebooks/1aki.pdb
2019-03-06 16:53:22,834 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'MODEL', 'ENDMDL']


In [5]:
#Show protein
view = nglview.show_file(downloaded_pdb_path)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

## Modeling the missing heavy atoms in the structure side chains
This BB will reconstruct missing atoms from residue side chains and detect all kinds of clashes

In [6]:
#Check & Fix PDB
#Import module
from biobb_model.model.fix_side_chain import FixSideChain
# Create prop dict and inputs/outputs
fixed_pdb_path = '1aki_fixed.pdb'
#Create and launch bb
FixSideChain(input_pdb_path=downloaded_pdb_path, output_pdb_path=fixed_pdb_path).launch()

=                     MDWeb structure checking utility                        =
=                      A. Hospital, J.L. Gelpi 2018                           =

Running fixside. Options: --fix All 
1aki.pdb
Structure 1aki.pdb loaded
Title: 
Experimental method: unknown
Resolution: 0.0 A

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  129
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  1001

No residues with missing side chain atoms found
Structure not modified, saving due to --force_save option
Structure saved on 1aki_fixed.pdb
Final Num. models: 1
Final Num. chains: 1 (A: Protein)
Final Num. residues:  129
Final Num. residues with ins. codes:  0
Final Num. HETATM residues:  0
Final Num. ligands or modified residues:  0
Final Num. water mol.:  0
Final Num. atoms:  1001



In [7]:
#Show protein
view = nglview.show_file(fixed_pdb_path)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

## Modeling mutation list
This BB will mutate existing residues to the requested ones.

In [8]:
#Mutate protein
# Import module to test
from biobb_model.model.mutate import Mutate
# Create prop dict and inputs/outputs
prop = {'mutation_list': 'Val2Tyr, Val2Ala'}
mutated_pdb_path = '1aki_mutated.pdb'
#Create and launch bb
Mutate(input_pdb_path=fixed_pdb_path,output_pdb_path=mutated_pdb_path, properties=prop).launch()

=                     MDWeb structure checking utility                        =
=                      A. Hospital, J.L. Gelpi 2018                           =

Running mutateside. Options: --mut Val2Tyr, Val2Ala 
1aki_fixed.pdb
Structure 1aki_fixed.pdb loaded
Title: 
Experimental method: unknown
Resolution: 0.0 A

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  129
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  1001

Mutations to perform
*:VAL2TYR
*:VAL2ALA
Replacing VAL A2 into TYR
  Renaming CG1 to CG
  Deleting CG2
  Adding new atom CD1
  Adding new atom CE1
  Adding new atom CZ
  Adding new atom OH
  Adding new atom CE2
  Adding new atom CD2

Replacing TYR A2 into ALA
  Deleting CG
  Deleting CD1
  Deleting CE1
  Deleting CZ
  Deleting CE2
  Deleting CD2
  Deleting OH

Checking new clashes
No severe clashes detected
No apolar clashes detected
No polar_acceptor clashes detected
No 

In [10]:
#Show log and protein
view1 = nglview.show_file(fixed_pdb_path)
view1.add_representation(repr_type='ball+stick', selection='2:A')
view1._remote_call('setSize', target='Widget', args=['500px','600px'])
view1.camera='orthographic'
view2 = nglview.show_file(mutated_pdb_path)
view2.add_representation(repr_type='ball+stick', selection='2:A')
view2._remote_call('setSize', target='Widget', args=['500px','600px'])
view2.camera='orthographic'
ipywidgets.HBox([view1, view2])

## Generate the topology
Generate the topology using [Pdb2gmx](http://manual.gromacs.org/programs/gmx-pdb2gmx.html) module.
The default forcefield is [amber99sb-ildn](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2970904/) and the default water_type is [spce](http://www.sklogwiki.org/SklogWiki/index.php/SPC/E_model_of_water).
This BB will generate 2 main files:
-  A compressed ZIP file containing:
> -  The [Gromacs topology file](http://manual.gromacs.org/online/top.html) (.top)
> -  The [Gromacs position restraint file](http://manual.gromacs.org/online/itp.html) (.itp)
-  A post-processed [Gromacs structure file](http://manual.gromacs.org/current/online/gro.html) (.gro)

In [11]:
#Create system topology
# Import module
from biobb_md.gromacs.pdb2gmx import Pdb2gmx
# Create inputs/outputs
output_pdb2gmx_gro_path = '1aki_pdb2gmx.gro'
output_pdb2gmx_top_zip_path = '1aki_pdb2gmx_top.zip'
#Create and launch bb
Pdb2gmx(input_pdb_path=mutated_pdb_path, output_gro_path=output_pdb2gmx_gro_path, output_top_zip_path=output_pdb2gmx_top_zip_path).launch()

2019-03-06 16:54:09,477 [MainThread  ] [INFO ]  GROMACS Pdb2gmx 20181 version detected
2019-03-06 16:54:09,839 [MainThread  ] [INFO ]  gmx pdb2gmx -f 1aki_mutated.pdb -o 1aki_pdb2gmx.gro -p p2g.top -water spce -ff amber99sb-ildn -i p2g.itp

2019-03-06 16:54:09,840 [MainThread  ] [INFO ]  Exit code 0

2019-03-06 16:54:09,841 [MainThread  ] [INFO ]  
Using the Amber99sb-ildn force field in directory amber99sb-ildn.ff

Reading 1aki_mutated.pdb...
Read 999 atoms
Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.
There are 1 chains and 0 blocks of water and 129 residues with 999 atoms

  chain  #res #atoms
  1 'A'   129    999  

Reading residue database... (amber99sb-ildn)
Processing chain 1 'A' (999 atoms, 129 residues)
Identified residue LYS1 as a starting terminus.
Identified residue LEU129 as a ending terminus.
Checking for duplicate atoms....
Generating any missing hydrogen atoms and/or adding termini.
Now there are 129 residues with 1954 atoms
		-

0

In [12]:
#Show protein
view = nglview.show_file(output_pdb2gmx_gro_path)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

Note that hydrogen atoms had been added to the structure.
***

## Create the solvent box
Create the solvent box using the [Editconf](http://manual.gromacs.org/programs/gmx-editconf.html) module. The box will be centered, the distance between the solute and the box will be 1.0nm and the box shape will be cubic by default. The main output of this BB will be an updated post-procesed Gromacs structure file (.gro). 

In [13]:
# Editconf: Create solvent box
# Import module
from biobb_md.gromacs.editconf import Editconf
# Create prop dict and inputs/outputs
output_editconf_gro_path = '1aki_editconf.gro'
#Create and launch bb
Editconf(input_gro_path=output_pdb2gmx_gro_path, output_gro_path=output_editconf_gro_path).launch()

2019-03-06 16:54:25,572 [MainThread  ] [INFO ]  GROMACS Editconf 20181 version detected
2019-03-06 16:54:25,573 [MainThread  ] [INFO ]  Centering molecule in the box.
2019-03-06 16:54:25,574 [MainThread  ] [INFO ]  Distance of the box to molecule:   1.00
2019-03-06 16:54:25,575 [MainThread  ] [INFO ]  Box type: cubic
2019-03-06 16:54:25,600 [MainThread  ] [INFO ]  gmx editconf -f 1aki_pdb2gmx.gro -o 1aki_editconf.gro -d 1.0 -bt cubic -c

2019-03-06 16:54:25,602 [MainThread  ] [INFO ]  Exit code 0

2019-03-06 16:54:25,603 [MainThread  ] [INFO ]  Read 1954 atoms
Volume: 55.8378 nm^3, corresponds to roughly 25100 electrons
No velocities found
    system size :  3.817  4.234  3.454 (nm)
    diameter    :  5.010               (nm)
    center      :  2.779  2.490  0.021 (nm)
    box vectors :  3.817  4.235  3.455 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :  55.84               (nm^3)
    shift       :  0.726  1.015  3.484 (nm)
new center      :  3.505  3.505  3.50

0

## Fill the solvent box with water molecules
Fill the solvent box using the Gromacs Solvate module. The "spc216.gro" will be default solvent model. The main output of this BB will be an updated post-procesed Gromacs structure file (.gro) and a zip file containing the updated topology file (.top) and the restriction files (.itp).

In [14]:
# Solvate: Fill the box with water molecules
from biobb_md.gromacs.solvate import Solvate
# Create prop dict and inputs/outputs
output_solvate_gro_path = '1aki_solvate.gro'
output_solvate_top_zip_path = '1aki_solvate_top.zip'
#Create and launch bb
Solvate(input_solute_gro_path=output_editconf_gro_path, output_gro_path=output_solvate_gro_path, input_top_zip_path=output_pdb2gmx_top_zip_path, output_top_zip_path=output_solvate_top_zip_path).launch()

2019-03-06 16:54:36,138 [MainThread  ] [INFO ]  GROMACS Solvate 20181 version detected
2019-03-06 16:54:36,146 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_wf_mutations/biobb_wf_mutations/notebooks/1aki_pdb2gmx_top.zip
2019-03-06 16:54:36,147 [MainThread  ] [INFO ]  to:
2019-03-06 16:54:36,148 [MainThread  ] [INFO ]  ['ea7cb3e4-1a43-46a9-936e-36051ea19cd0/p2g.itp', 'ea7cb3e4-1a43-46a9-936e-36051ea19cd0/p2g.top']
2019-03-06 16:54:36,149 [MainThread  ] [INFO ]  Unzipping: 
2019-03-06 16:54:36,150 [MainThread  ] [INFO ]  1aki_pdb2gmx_top.zip
2019-03-06 16:54:36,152 [MainThread  ] [INFO ]  To: 
2019-03-06 16:54:36,153 [MainThread  ] [INFO ]  ea7cb3e4-1a43-46a9-936e-36051ea19cd0/p2g.itp
2019-03-06 16:54:36,154 [MainThread  ] [INFO ]  ea7cb3e4-1a43-46a9-936e-36051ea19cd0/p2g.top
2019-03-06 16:54:36,378 [MainThread  ] [INFO ]  gmx solvate -cp 1aki_editconf.gro -cs spc216.gro -o 1aki_solvate.gro -p ea7cb3e4-1a43-46a9-936e-36051ea19cd0/p2g.top

2019-03-06 16:54:36,380 [MainThre

0

In [15]:
#Show protein
view = nglview.show_file(output_solvate_gro_path)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='green')
view.add_representation(repr_type='ball+stick', selection='SOL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

## Preprocess ion generation
Create the [portable binary run file (.tpr)](http://manual.gromacs.org/online/tpr.html) for ion generation, using the Grompp module. The main default parameters for this execution are:
-  integrator  = steep         ; Algorithm (steep = steepest descent minimization)
-  emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
-  emstep      = 0.01          ; Minimization step size
-  nsteps      = 50000         ; Maximum number of (minimization) steps to perform

The main output of this BB will be the portable binary run file (.tpr).

In [16]:
# Grompp: Creating portable binary run file for ion generation
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'minimization', 'nsteps':'5000'}}
output_gppion_tpr_path = '1aki_gppion.tpr'
#Create and launch bb
Grompp(input_gro_path=output_solvate_gro_path, input_top_zip_path=output_solvate_top_zip_path, output_tpr_path=output_gppion_tpr_path,  properties=prop).launch()

2019-03-06 16:54:43,491 [MainThread  ] [INFO ]  GROMACS Grompp 20181 version detected
2019-03-06 16:54:43,494 [MainThread  ] [INFO ]  Will run a minimization md of 5000 steps
2019-03-06 16:54:43,500 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_wf_mutations/biobb_wf_mutations/notebooks/1aki_solvate_top.zip
2019-03-06 16:54:43,501 [MainThread  ] [INFO ]  to:
2019-03-06 16:54:43,503 [MainThread  ] [INFO ]  ['77f2a9c9-ae1f-4cfb-a0bc-3ee6c6eec8b5/p2g.itp', '77f2a9c9-ae1f-4cfb-a0bc-3ee6c6eec8b5/p2g.top']
2019-03-06 16:54:43,505 [MainThread  ] [INFO ]  Unzipping: 
2019-03-06 16:54:43,506 [MainThread  ] [INFO ]  1aki_solvate_top.zip
2019-03-06 16:54:43,506 [MainThread  ] [INFO ]  To: 
2019-03-06 16:54:43,508 [MainThread  ] [INFO ]  77f2a9c9-ae1f-4cfb-a0bc-3ee6c6eec8b5/p2g.itp
2019-03-06 16:54:43,509 [MainThread  ] [INFO ]  77f2a9c9-ae1f-4cfb-a0bc-3ee6c6eec8b5/p2g.top
2019-03-06 16:54:43,796 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1aki_solvate.gro -r 1aki_solvate.gr

0

## Ion generation
Replace solvent molecules to neutralice the system and then reach a 0.05 mol/litre concentration by default. Using the Genion module.
The main output of this BB will be an updated post-procesed Gromacs structure file (.gro) and a zip file containing the updated topology file (.top) and the restriction files (.itp).

In [17]:
# Genion: Adding ions to reach a 0.05 nm concentration
from biobb_md.gromacs.genion import Genion
# Create prop dict and inputs/outputs
prop={'neutral':True, 'concentration':0.05}
output_genion_gro_path = '1aki_genion.gro'
output_genion_top_zip_path = '1aki_genion_top.zip'
#Create and launch bb
Genion(input_tpr_path=output_gppion_tpr_path, output_gro_path=output_genion_gro_path, input_top_zip_path=output_solvate_top_zip_path, output_top_zip_path=output_genion_top_zip_path, properties=prop).launch()

2019-03-06 16:54:46,828 [MainThread  ] [INFO ]  GROMACS Genion 20181 version detected
2019-03-06 16:54:46,830 [MainThread  ] [INFO ]  To reach up 0.05 mol/litre concentration
2019-03-06 16:54:46,837 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_wf_mutations/biobb_wf_mutations/notebooks/1aki_solvate_top.zip
2019-03-06 16:54:46,838 [MainThread  ] [INFO ]  to:
2019-03-06 16:54:46,839 [MainThread  ] [INFO ]  ['f73835ca-271f-4068-9b2f-118c2889fd0b/p2g.itp', 'f73835ca-271f-4068-9b2f-118c2889fd0b/p2g.top']
2019-03-06 16:54:46,840 [MainThread  ] [INFO ]  Unzipping: 
2019-03-06 16:54:46,841 [MainThread  ] [INFO ]  1aki_solvate_top.zip
2019-03-06 16:54:46,842 [MainThread  ] [INFO ]  To: 
2019-03-06 16:54:46,843 [MainThread  ] [INFO ]  f73835ca-271f-4068-9b2f-118c2889fd0b/p2g.itp
2019-03-06 16:54:46,844 [MainThread  ] [INFO ]  f73835ca-271f-4068-9b2f-118c2889fd0b/p2g.top
2019-03-06 16:54:46,961 [MainThread  ] [INFO ]  echo "SOL" | gmx genion -s 1aki_gppion.tpr -o 1aki_genion.gro -

0

In [18]:
#Show protein
view = nglview.show_file(output_genion_gro_path)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='ball+stick', selection='NA')
view.add_representation(repr_type='ball+stick', selection='CL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

## Preprocess energy minimization
Create the [portable binary run file (.tpr)](http://manual.gromacs.org/online/tpr.html) for energy minimization, using the Grompp module. The main default parameters for this execution are:
-  integrator  = steep         ; Algorithm (steep = steepest descent minimization)
-  emtol       = 500.0         ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
-  emstep      = 0.01          ; Minimization step size
-  nsteps      = 50000         ; Maximum number of (minimization) steps to perform

The main output of this BB will be the portable binary run file (.tpr).

In [19]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'minimization', 'nsteps':'5000', 'emtol':'500'}}
output_gppmin_tpr_path = '1aki_gppmin.tpr'
#Create and launch bb
Grompp(input_gro_path=output_genion_gro_path, input_top_zip_path=output_genion_top_zip_path, output_tpr_path=output_gppmin_tpr_path,  properties=prop).launch()

2019-03-06 16:54:50,703 [MainThread  ] [INFO ]  GROMACS Grompp 20181 version detected
2019-03-06 16:54:50,706 [MainThread  ] [INFO ]  Will run a minimization md of 5000 steps
2019-03-06 16:54:50,713 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_wf_mutations/biobb_wf_mutations/notebooks/1aki_genion_top.zip
2019-03-06 16:54:50,714 [MainThread  ] [INFO ]  to:
2019-03-06 16:54:50,715 [MainThread  ] [INFO ]  ['17c6538d-9274-4563-b477-29066e09f05f/p2g.itp', '17c6538d-9274-4563-b477-29066e09f05f/p2g.top']
2019-03-06 16:54:50,717 [MainThread  ] [INFO ]  Unzipping: 
2019-03-06 16:54:50,718 [MainThread  ] [INFO ]  1aki_genion_top.zip
2019-03-06 16:54:50,719 [MainThread  ] [INFO ]  To: 
2019-03-06 16:54:50,720 [MainThread  ] [INFO ]  17c6538d-9274-4563-b477-29066e09f05f/p2g.itp
2019-03-06 16:54:50,722 [MainThread  ] [INFO ]  17c6538d-9274-4563-b477-29066e09f05f/p2g.top
2019-03-06 16:54:50,988 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1aki_genion.gro -r 1aki_genion.gro -p

0

## Execute system equilibration
Execute the energy minimization using the MDrun module and the input the portable binary run file (.tpr) as the main input.
The main output of this BB will be updated structure file (.gro) and the [Gromacs trajectory file](http://manual.gromacs.org/online/trr.html) (.trr).

In [20]:
# Mdrun: Running minimization
from biobb_md.gromacs.mdrun import Mdrun
# Create prop dict and inputs/outputs
output_min_trr_path = '1aki_min.trr'
output_min_gro_path = '1aki_min.gro'
output_min_edr_path = '1aki_min.edr'
output_min_log_path = '1aki_min.log'
#Create and launch bb
Mdrun(input_tpr_path=output_gppmin_tpr_path, output_trr_path=output_min_trr_path, output_gro_path=output_min_gro_path, output_edr_path=output_min_edr_path, output_log_path=output_min_log_path).launch()

2019-03-06 16:54:52,438 [MainThread  ] [INFO ]  GROMACS Mdrun 20181 version detected
2019-03-06 16:55:43,247 [MainThread  ] [INFO ]  gmx mdrun -s 1aki_gppmin.tpr -o 1aki_min.trr -c 1aki_min.gro -e 1aki_min.edr -g 1aki_min.log -nt 0

2019-03-06 16:55:43,249 [MainThread  ] [INFO ]  Exit code 0

2019-03-06 16:55:43,249 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Mi

0

## Preprocess system temperature equilibration
Equilibrate the solvent and ions restraining the protein heavy atoms
Create the [portable binary run file (.tpr)](http://manual.gromacs.org/online/tpr.html) for system equilibration, using the Grompp module. The main default parameters for this execution are:
-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000
-  pcoupl                   = no
-  gen_vel                  = yes
-  gen_temp                 = 300
-  gen_seed                 = -1

The main output of this BB will be the portable binary run file (.tpr).

In [21]:
# Grompp: Creating portable binary run file for Equilibration
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'nvt', 'nsteps':'5000'}}
output_gppnvt_tpr_path = '1aki_gppnvt.tpr'
#Create and launch bb
Grompp(input_gro_path=output_min_gro_path, input_top_zip_path=output_genion_top_zip_path, output_tpr_path=output_gppnvt_tpr_path,  properties=prop).launch()

2019-03-06 16:55:43,285 [MainThread  ] [INFO ]  GROMACS Grompp 20181 version detected
2019-03-06 16:55:43,289 [MainThread  ] [INFO ]  Will run a nvt md of 5000 steps
2019-03-06 16:55:43,298 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_wf_mutations/biobb_wf_mutations/notebooks/1aki_genion_top.zip
2019-03-06 16:55:43,299 [MainThread  ] [INFO ]  to:
2019-03-06 16:55:43,300 [MainThread  ] [INFO ]  ['9501fb57-c2e1-4458-a151-031e02d2c71f/p2g.itp', '9501fb57-c2e1-4458-a151-031e02d2c71f/p2g.top']
2019-03-06 16:55:43,302 [MainThread  ] [INFO ]  Unzipping: 
2019-03-06 16:55:43,303 [MainThread  ] [INFO ]  1aki_genion_top.zip
2019-03-06 16:55:43,305 [MainThread  ] [INFO ]  To: 
2019-03-06 16:55:43,306 [MainThread  ] [INFO ]  9501fb57-c2e1-4458-a151-031e02d2c71f/p2g.itp
2019-03-06 16:55:43,308 [MainThread  ] [INFO ]  9501fb57-c2e1-4458-a151-031e02d2c71f/p2g.top
2019-03-06 16:55:43,804 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1aki_min.gro -r 1aki_min.gro -p 9501fb57-c2e1-

0

## Execute system temperature equilibration
Execute the system equilibration using the MDrun module and the input the portable binary run file (.tpr) as the main input.
The main output of this BB will be updated structure file (.gro) and the [Gromacs trajectory file](http://manual.gromacs.org/online/trr.html) (.trr).

In [43]:
# Mdrun: Running Equilibration NVT
from biobb_md.gromacs.mdrun import Mdrun
# Create prop dict and inputs/outputs
output_nvt_trr_path = '1aki_nvt.trr'
output_nvt_gro_path = '1aki_nvt.gro'
output_nvt_edr_path = '1aki_nvt.edr'
output_nvt_log_path = '1aki_nvt.log'
output_nvt_cpt_path = '1aki_nvt.cpt'
#Create and launch bb
Mdrun(input_tpr_path=output_gppnvt_tpr_path, output_trr_path=output_nvt_trr_path, output_gro_path=output_nvt_gro_path, output_edr_path=output_nvt_edr_path, output_log_path=output_nvt_log_path, output_cpt_path=output_nvt_cpt_path).launch()

2019-01-24 11:06:32,271 [MainThread  ] [INFO ]  GROMACS Mdrun 20181 version detected
2019-01-24 11:07:19,722 [MainThread  ] [INFO ]  gmx mdrun -s 1aki_gppnvt.tpr -o 1aki_nvt.trr -c 1aki_nvt.gro -e 1aki_nvt.edr -g 1aki_nvt.log -nt 0 -cpo 1aki_nvt.cpt

2019-01-24 11:07:19,724 [MainThread  ] [INFO ]  Exit code 0

2019-01-24 11:07:19,725 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexe

0

## Preprocess system pressure equilibration
Equilibrate the solvent and ions restraining the protein heavy atoms.
Create the [portable binary run file (.tpr)](http://manual.gromacs.org/online/tpr.html) for system equilibration, using the Grompp module. The main default parameters for this execution are:
-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000
-  pcoupl = Parrinello-Rahman
-  pcoupltype = isotropic
-  tau_p = 1.0
-  ref_p = 1.0
-  compressibility = 4.5e-5
-  refcoord_scaling = com
-  gen_vel = no

The main output of this BB will be the portable binary run file (.tpr).

In [44]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'npt', 'nsteps':'5000'}}
output_gppnpt_tpr_path = '1aki_gppnpt.tpr'
#Create and launch bb
Grompp(input_gro_path=output_nvt_gro_path, input_top_zip_path=output_genion_top_zip_path, output_tpr_path=output_gppnpt_tpr_path, input_cpt_path=output_nvt_cpt_path,  properties=prop).launch()

2019-01-24 11:07:19,758 [MainThread  ] [INFO ]  GROMACS Grompp 20181 version detected
2019-01-24 11:07:19,761 [MainThread  ] [INFO ]  Will run a npt md of 5000 steps
2019-01-24 11:07:19,768 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_wf_mutations/biobb_wf_mutations/notebooks/1aki_genion_top.zip
2019-01-24 11:07:19,769 [MainThread  ] [INFO ]  to:
2019-01-24 11:07:19,770 [MainThread  ] [INFO ]  ['1444c7db-b3eb-4301-af17-20ea72ce1515/p2g.itp', '1444c7db-b3eb-4301-af17-20ea72ce1515/p2g.top']
2019-01-24 11:07:19,771 [MainThread  ] [INFO ]  Unzipping: 
2019-01-24 11:07:19,772 [MainThread  ] [INFO ]  1aki_genion_top.zip
2019-01-24 11:07:19,773 [MainThread  ] [INFO ]  To: 
2019-01-24 11:07:19,774 [MainThread  ] [INFO ]  1444c7db-b3eb-4301-af17-20ea72ce1515/p2g.itp
2019-01-24 11:07:19,775 [MainThread  ] [INFO ]  1444c7db-b3eb-4301-af17-20ea72ce1515/p2g.top
2019-01-24 11:07:20,276 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1aki_nvt.gro -r 1aki_nvt.gro -p 1444c7db-b3eb-

0

## Execute system pressure equilibration
Execute the system equilibration using the MDrun module and the input the portable binary run file (.tpr) as the main input.
The main output of this BB will be updated structure file (.gro) and the [Gromacs trajectory file](http://manual.gromacs.org/online/trr.html) (.trr).

In [45]:
# Mdrun: Running minimization NPT
from biobb_md.gromacs.mdrun import Mdrun
# Create prop dict and inputs/outputs
output_npt_trr_path = '1aki_npt.trr'
output_npt_gro_path = '1aki_npt.gro'
output_npt_edr_path = '1aki_npt.edr'
output_npt_log_path = '1aki_npt.log'
output_npt_cpt_path = '1aki_npt.cpt'
#Create and launch bb
Mdrun(input_tpr_path=output_gppnpt_tpr_path, output_trr_path=output_npt_trr_path, output_gro_path=output_npt_gro_path, output_edr_path=output_npt_edr_path, output_log_path=output_npt_log_path, output_cpt_path=output_npt_cpt_path).launch()

2019-01-24 11:07:20,312 [MainThread  ] [INFO ]  GROMACS Mdrun 20181 version detected
2019-01-24 11:08:07,446 [MainThread  ] [INFO ]  gmx mdrun -s 1aki_gppnpt.tpr -o 1aki_npt.trr -c 1aki_npt.gro -e 1aki_npt.edr -g 1aki_npt.log -nt 0 -cpo 1aki_npt.cpt

2019-01-24 11:08:07,448 [MainThread  ] [INFO ]  Exit code 0

2019-01-24 11:08:07,449 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexe

0

## Preprocess free dynamics
Free molecular dynamics simulation.
Create the [portable binary run file (.tpr)](http://manual.gromacs.org/online/tpr.html) for free molecular dynamics simulation, using the Grompp module. The main default parameters for this execution are:
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000

The main output of this BB will be the portable binary run file (.tpr).

In [46]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'free', 'nsteps':'15000'}}
output_gppmd_tpr_path = '1aki_gppmd.tpr'
#Create and launch bb
Grompp(input_gro_path=output_npt_gro_path, input_top_zip_path=output_genion_top_zip_path, output_tpr_path=output_gppmd_tpr_path, input_cpt_path=output_npt_cpt_path, properties=prop).launch()

2019-01-24 11:08:07,481 [MainThread  ] [INFO ]  GROMACS Grompp 20181 version detected
2019-01-24 11:08:07,483 [MainThread  ] [INFO ]  Will run a free md of 30.0 pico seconds
2019-01-24 11:08:07,489 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_wf_mutations/biobb_wf_mutations/notebooks/1aki_genion_top.zip
2019-01-24 11:08:07,490 [MainThread  ] [INFO ]  to:
2019-01-24 11:08:07,491 [MainThread  ] [INFO ]  ['865ca228-1f37-46e9-81c7-407b87206990/p2g.itp', '865ca228-1f37-46e9-81c7-407b87206990/p2g.top']
2019-01-24 11:08:07,492 [MainThread  ] [INFO ]  Unzipping: 
2019-01-24 11:08:07,493 [MainThread  ] [INFO ]  1aki_genion_top.zip
2019-01-24 11:08:07,494 [MainThread  ] [INFO ]  To: 
2019-01-24 11:08:07,495 [MainThread  ] [INFO ]  865ca228-1f37-46e9-81c7-407b87206990/p2g.itp
2019-01-24 11:08:07,496 [MainThread  ] [INFO ]  865ca228-1f37-46e9-81c7-407b87206990/p2g.top
2019-01-24 11:08:07,872 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1aki_npt.gro -r 1aki_npt.gro -p 865ca2

0

## Execute free molecular dynamics simulation
Execute the free molecular dynamics simulation  using the MDrun module and the input the portable binary run file (.tpr) as the main input.
The main output of this BB will be updated structure file (.gro) and the [Gromacs trajectory file](http://manual.gromacs.org/online/trr.html) (.trr).

In [47]:
# Mdrun: Running free dynamics
from biobb_md.gromacs.mdrun import Mdrun
# Create prop dict and inputs/outputs
output_md_trr_path = '1aki_md.trr'
output_md_gro_path = '1aki_md.gro'
output_md_edr_path = '1aki_md.edr'
output_md_log_path = '1aki_md.log'
output_md_cpt_path = '1aki_md.cpt'
#Create and launch bb
Mdrun(input_tpr_path=output_gppmd_tpr_path, output_trr_path=output_md_trr_path, output_gro_path=output_md_gro_path, output_edr_path=output_md_edr_path, output_log_path=output_md_log_path, output_cpt_path=output_md_cpt_path).launch()

2019-01-24 11:08:07,907 [MainThread  ] [INFO ]  GROMACS Mdrun 20181 version detected
2019-01-24 11:10:27,472 [MainThread  ] [INFO ]  gmx mdrun -s 1aki_gppmd.tpr -o 1aki_md.trr -c 1aki_md.gro -e 1aki_md.edr -g 1aki_md.log -nt 0 -cpo 1aki_md.cpt

2019-01-24 11:10:27,474 [MainThread  ] [INFO ]  Exit code 0

2019-01-24 11:10:27,475 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shve

0

In [ ]:
#Show protein
nglview.show_simpletraj(nglview.SimpletrajTrajectory(output_md_trr_path, output_md_gro_path), gui=True)

In [24]:
#Clean the directory Remove everything
!find . ! \( -iname "*.ipynb" \) -exec rm -rf {} 2>/dev/null \; -print

./1aki_npt.edr
./1aki_min.log
./1aki_gppmin.tpr
./1aki_npt.trr
./1aki_md.log
./1aki_npt.cpt
./1aki_genion.gro
./1aki_md.gro
./1aki_editconf.gro
./1aki_nvt.edr
./1aki_fixed.pdb
./1aki.pdb
./1aki_solvate.gro
./1aki_min.gro
./1aki_nvt.trr
./1aki_nvt.cpt
./1aki_gppion.tpr
./1aki_min.trr
./log.out
./1aki_nvt.gro
./1aki_md.edr
./1aki_gppmd.tpr
./1aki_mutated.pdb
./1aki_solvate_top.zip
./1aki_genion_top.zip
./1aki_min.edr
./1aki_md.trr
./1aki_npt.log
./1aki_gppnpt.tpr
./1aki_md.cpt
./.ipynb_checkpoints
./1aki_npt.gro
./1aki_nvt.log
./1aki_pdb2gmx_top.zip
./1aki_gppnvt.tpr
./1aki_pdb2gmx.gro
./log.err
